In [1]:
import pandas as pd
import numpy as np
import math
import itertools
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import time
from prophet import Prophet
import matplotlib.pyplot as plt
import joblib

c:\Users\Harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_data_df = pd.read_excel('C:/Users/Harsh/Desktop/trainer/data/Cauvery.xlsx') # creates a Pandas data frame for input value
	    


In [7]:

raw_data_df['Date']=pd.to_datetime(raw_data_df['Date'])

for i in range(1,len(raw_data_df.columns)):
	raw_data_df[raw_data_df.columns[i]] = raw_data_df[raw_data_df.columns[i]].fillna(raw_data_df[raw_data_df.columns[i]].mean())

data=pd.DataFrame()

data['Date']=raw_data_df["Date"]
data['weekly runoff']=raw_data_df["weekly runoff"]
data=data.set_index(['Date'])
# In[5]:
data.head()
# In[7]:
data.isnull().sum()

weekly runoff    0
dtype: int64

In [8]:
monthly = data.resample('M').sum()

In [9]:
weekly = data.resample('W').sum()
daily = data.resample('D').sum()
#daily.rolling(30, center=True).sum().plot(style=[':', '--', '-'],title='Daily Trends')

daily.head()

,weekly runoff
Date,
1998-01-01,7.87895
1998-01-02,7.87895
1998-01-03,7.87895
1998-01-04,7.87895
1998-01-05,7.87895


In [10]:
#Use MinMaxScaler to normalize  to range from 0 to 1
values = daily['weekly runoff'].values.reshape(-1,1)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# In[27]:
scale=daily
scale["weekly runoff"]=scaled
scale.head()
# In[28]:
scale.shape
# In[38]:


(7670, 1)

In [11]:
def making_dataset(i=1):
    #Testing the future prediction
    if i==0 :
        #Taking data of last two years as testing data
        df1=scale.iloc[6940:,:]
        #Training Data
        df2=scale.iloc[:6940,:]
        df2.reset_index(inplace=True)
        # Prophet requires columns ds (Date) and y (value)
        df2 = df2.rename(columns={'Date': 'ds', 'weekly runoff': 'y'})
        return df1,df2
    else:
        #Predicting the future values after 2018
        df2=scale.iloc[:,:]
        df2.reset_index(inplace=True)
        # Prophet requires columns ds (Date) and y (value)
        df2 = df2.rename(columns={'Date': 'ds', 'weekly runoff': 'y'})
        return df2,df2

In [12]:
df1,df2=making_dataset(1)
df2.head()
# In[40]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
df2_prophet = Prophet(changepoint_prior_scale=0.05)
df2_prophet.fit(df2)
path='C:/Users/Harsh/Desktop/trainer/Models'
joblib.dump(df2_prophet, path+'.pkl')
warnings.resetwarnings()

21:11:13 - cmdstanpy - INFO - Chain [1] start processing
21:11:15 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
def predicting_data(i=1):
    if i==0:
        #For testing 
        # Make a future dataframe for (2 Years)
        df2_forecast = df2_prophet.make_future_dataframe(periods=30*25 , freq='D')
        # Make predictions
        df2_forecast = df2_prophet.predict(df2_forecast)
        df3=df2_forecast[['ds','yhat']]
        df3.shape,df1.shape,df2.shape
        df4=df3.iloc[6940:-20,:]

    else:
        #For Future prediction of 2019
        # Make a future dataframe for 12 months
        df2_forecast = df2_prophet.make_future_dataframe(periods=30*12 , freq='D',include_history=False)
        # Make predictions
        df2_forecast = df2_prophet.predict(df2_forecast)
        df3=df2_forecast[['ds','yhat']]
        #df3.shape,df1.shape,df2.shape
        df4=df3.iloc[:,:]
    return df4,df2_forecast

In [16]:
df4,df2_forecast=predicting_data(1)
ypred=df4.iloc[:,1:]
ytest=df1.iloc[:,:]
ypred.shape,ytest.shape

df4.tail()
# In[52]:
ypred=df4.iloc[:,1:]
ytest=df1.iloc[:,:]
ypred.shape,ytest.shape

((360, 1), (7670, 2))